In [12]:
import os

import pyarrow as pa
import pyarrow.parquet as pq
import pandas as pd

In [8]:
df1 = pd.DataFrame([
    {'i': 1, 'f': 1.1, 's': 'abc'},
    {'i': 1, 's': 'abc'},
])
df2 = pd.DataFrame([
    {'i': 2, 'f': 1.1, 's': 'abcd'},
    {'i': 2, 's': 'abcd'},
])
csv1 = df1.to_csv(index=False)
csv2 = df2.to_csv(index=False)

In [18]:
data_dir = '/home/matthew/Documents/TSE/AppliedEconometrics/repo/data'
source_path = os.path.join(data_dir, 'debug/DISPATCHLOAD.parquet')
dest_path = os.path.join(data_dir, 'debug/DISPATCHLOAD-pq/')
with pq.ParquetFile(source_path) as f_in:
    
    for batch in f_in.iter_batches(use_threads = False):
        pq.write_to_dataset(
            table = batch,
            root_path = dest_path,
            partitioning = ['DUID'],
            use_threads = False, # lower memory
        )

KeyboardInterrupt: 

In [ ]:
data_dir = '/home/matthew/Documents/TSE/AppliedEconometrics/repo/data'
source_path = os.path.join(data_dir, 'debug/DISPATCHLOAD.parquet')
dest_path = os.path.join(data_dir, 'debug/DISPATCHLOAD-pq-2/')
with pq.ParquetFile(source_path) as f_in:
    batches = f_in.iter_batches(use_threads = False)
    table = pa.Table.from_batches(batches, schema=f_in.schema_arrow)
    pq.write_to_dataset(
        table = f_in,
        root_path = dest_path,
        partitioning = ['DUID'],
        schema = f_in.schema_arrow,
        use_threads = False, # lower memory
    )

In [ ]:
with pyarrow.parquet.ParquetWriter(parquet_file, output_schema) as writer:
        for csv_file in os.listdir(csv_folder):
            try:
                csv_path = os.path.join(csv_folder, csv_file)
                csv_reader = pyarrow.csv.open_csv(csv_path, 
                                                  convert_options=pyarrow.csv.ConvertOptions(
                                                      column_types=input_schema, 
                                                      include_missing_columns=True, 
                                                      include_columns=column_names))
    
                for batch in csv_reader:
                    check_for_pause()
                    columns = []
                    if datetime_columns == []:
                        # table has no datetime columns
                        # just write straight into the parquet file
                        writer.write(batch)
                    else:
                        # parse some columns from string to datetime
                        
                        for (i, column_name) in enumerate(column_names):
                            assert isinstance(column_name, str), f"column_name is type {type(column_name)}"
                            column_data = batch[column_name]
                            if column_name in datetime_columns:
                                # pyarrow assumes the timesstamp is UTC by default
                                # the schema specifies the timezone as UTC+10 (which we want)
                                # but it adds 10h when doing the conversion
                                # so let's subtract 10h
                                # This is checked with a unit test later
                                column_data = pc.strptime(column_data, format="%Y/%m/%d %H:%M:%S", unit="s", error_is_null=True)
                                column_data = pc.add(column_data, -dt.timedelta(hours=TIMEZONE_OFFSET))
                            columns.append(column_data)
                        
                        updated_table = pyarrow.Table.from_arrays(
                            columns, 
                            schema=output_schema
                        )
                        writer.write(updated_table)
            except Exception:
                # additional logging
                logger.error(f"Error with file {csv_path}")
                raise